In [ ]:
# Ejemplo completo de un Perceptron en Pytorch
# Se observa el gradiente y las actualizaciones de los pesos
# Implementación elemental de Pytorch.

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

### Ejemplo 1: elemental

In [53]:
# Defino una matriz X (3,2) = (batch_size, input_size=no_features)
X = torch.tensor([[1,2], [3,4], [5,6]], dtype=torch.float)

# X = 	[1,2
#		 3,4
#		 5,6]

# Defino W y B para mi layer. 
# W es una matriz (2,1) y B es un vector (1)
# (2,1) = (input_size, output_size=n_neuronas)
# (1) = (output_size=n_neuronas)

W = torch.tensor([[0.5], [-0.3]])
B = torch.tensor([0.1]) # pytorch hace la suma element-wise automaticamente

# W = 	[0.5
#		-0.3]

# B = 	[0.1]

# Lo que hace el proceso es logit = X*W + B

logit = torch.matmul(X, W) + B
print(logit)

#logit = [0
#		  0.4
#		  0.8]
# estos son, para el batch, los valores predichos.

tensor([[-2.2352e-08],
        [ 4.0000e-01],
        [ 8.0000e-01]])


In [54]:
# check in pytorch
linear	= nn.Linear(2, 1) # (input_size, output_size)
linear.weight.data = W.T
linear.bias.data = B

linear(X)

tensor([[-2.2352e-08],
        [ 4.0000e-01],
        [ 8.0000e-01]], grad_fn=<AddmmBackward0>)

### Ejemplo 2: calculo de perdida

In [55]:
# Defino una matriz X (3,2) = (batch_size, input_size=no_features)
X = torch.tensor([[1,2], [3,4], [5,6]], dtype=torch.float)

# X = 	[1,2
#		 3,4
#		 5,6]

# Defino W y B para mi layer. 
# W es una matriz (2,1) y B es un vector (1)
# (2,1) = (input_size, output_size=n_neuronas)
# (1) = (output_size=n_neuronas)

W = torch.tensor([[0.5], [-0.3]])
B = torch.tensor([0.1]) # pytorch hace la suma element-wise automaticamente

# W = 	[0.5
#		-0.3]

# B = 	[0.1]

y = torch.tensor([[1], [0], [1]], dtype=torch.float)


In [56]:
# 1. Construyo señal neta (=logit).
# Lo que hace el proceso es Y_est = X*W + B
# El resultado son logits, valores brutos entre [-inf, inf]
logits = torch.matmul(X, W) + B
print(logits)

#sn = [0
#		  0.4
#		  0.8]
# estos son, para el batch, los valores predichos.

tensor([[-2.2352e-08],
        [ 4.0000e-01],
        [ 8.0000e-01]])


In [57]:
# 2. Aplico función de activación
y_probs = torch.sigmoid(logits)
y_probs

# alternativa
# s_layer = nn.Sigmoid()
# y_probs = s_layer.forward(logits)
# y_probs

tensor([[0.5000],
        [0.5987],
        [0.6900]])

In [58]:
# Que es lo mismo que hacerlo a mano 1/(1 + exp(-sn))
1/(1 + torch.exp(-logits))

tensor([[0.5000],
        [0.5987],
        [0.6900]])

In [59]:
# calculo perdida
# Se calcula entropia cruzada binaria.
# bce se aplica sobre y_probs, no sobre los logits.
# Esto es porque aplica sobre valores entre [0,1] y no sobre valores brutos.
# Fijarse que este caso donde el batch es mayor a uno, el valor es el promedio
# de las entropias cruzadas de cada elemento del batch.

bce_manual = -torch.mean(y * torch.log(y_probs) + (1 - y) * torch.log(1 - y_probs))
bce_manual

tensor(0.6591)

In [60]:
# ojo BCEWithLogitsLoss aplica ambas funciones, sigmoid y BCE
# esto permite aplicarlo directamente sobre los logits.
loss_fn = nn.BCEWithLogitsLoss()
loss_fn(logits, y)

tensor(0.6591)

### Ejemplo 3: comportamiento del gradiente

In [61]:
# Defino una matriz X (3,2) = (batch_size, input_size=no_features)
X = torch.tensor([[1,2], [3,4], [5,6]], dtype=torch.float)
y = torch.tensor([[1], [0], [1]], dtype=torch.float)

W = torch.tensor([[0.5], [-0.3]])
B = torch.tensor([0.1]) # pytorch hace la suma element-wise automaticamente

# X = 	[1,2
#		 3,4
#		 5,6]

# Defino W y B para mi layer. 
# W es una matriz (2,1) y B es un vector (1)
# (2,1) = (input_size, output_size=n_neuronas)
# (1) = (output_size=n_neuronas)

# W = 	[0.5
#		-0.3]

# B = 	[0.1]

# modelo
linear	= nn.Linear(2, 1) # (input_size, output_size)
linear.weight.data = W.T
linear.bias.data = B

# loss y optimizador
loss_fn = nn.BCEWithLogitsLoss() # BCE
optimizer = optim.SGD(linear.parameters(), lr=0.01) # Stochoastic Gradient Descent.

# resultado
logits = linear(X)
print(logits)

# perdida
loss = loss_fn(logits, y)
print(loss)

tensor([[-2.2352e-08],
        [ 4.0000e-01],
        [ 8.0000e-01]], grad_fn=<AddmmBackward0>)
tensor(0.6591, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


In [62]:
# gradiente
# estos son los valores del gradiente: dL/dW y dL/dB
# que se calculan con backpropagation.
optimizer.zero_grad() # limpio gradientes
loss.backward() # backpropagation

print(linear.weight.grad)
print(linear.bias.grad)

tensor([[-0.0847, -0.1551]])
tensor([-0.0704])


In [63]:
# calculo a mano de gradiente dL/dW
# dL/dW = X^T * (sigmoid(logits) - y) / batch_size
grad_dl_dw = torch.matmul(X.T, torch.sigmoid(logits) - y) / y.size(0)
grad_dl_dw

tensor([[-0.0847],
        [-0.1551]], grad_fn=<DivBackward0>)

In [64]:
# calculo a mano de gradiente dL/dB
# dL/dB = sum(sigmoid(logits) - y) / batch_size
grad_b_w = torch.mean((torch.sigmoid(logits) - y))
grad_b_w

tensor(-0.0704, grad_fn=<MeanBackward0>)

### Ejemplo 4: aplicacion del gradiente

In [65]:
lr=0.01

new_W = linear.weight.data.T -lr*grad_dl_dw
new_W

tensor([[ 0.5008],
        [-0.2984]], grad_fn=<SubBackward0>)

In [66]:
new_B = linear.bias.data - lr*grad_b_w
new_B

tensor([0.1007], grad_fn=<SubBackward0>)

In [67]:
optimizer.step() # actualizo W y B
print(linear.weight.data)
print(linear.bias.data)

# originales
# W = torch.tensor([[0.5], [-0.3]])
# B = torch.tensor([0.1])

tensor([[ 0.5008, -0.2984]])
tensor([0.1007])


### Ejemplo 5: implementación completa en Pytorch

In [68]:
# X in (3,2), y in (3,1)
X = torch.tensor([[1,2], [3,4], [5,6]], dtype=torch.float)
y = torch.tensor([[1], [0], [1]], dtype=torch.float)

In [69]:
class Perceptron(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)
        
    def forward(self, x):
        return self.layer(x)
    

# Init,loss and optimizer
model = Perceptron()
loss = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [70]:
# training loop
epochs = 5
for epoch in range(epochs):
    optimizer.zero_grad()
    logits = model(X)
    loss_values = loss(logits, y)
    loss_values.backward()
    optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss_values.item():.4f}")
    

Epoch 1, Loss: 2.1722
Epoch 2, Loss: 2.0730
Epoch 3, Loss: 1.9755
Epoch 4, Loss: 1.8799
Epoch 5, Loss: 1.7866
